In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os
from datetime import datetime
import logging

import matplotlib.pyplot as plt


from decimal import Decimal
from tinkoff.invest import MoneyValue
from tinkoff.invest.sandbox.client import SandboxClient

sys.path.append("..") 
from tp_utils import *
from tp_config import *
import tink_port as tink


pd.set_option("display.max_rows", 90)

In [2]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'
SANDBOX = 't.ZsHS2K9VKfl4Od15pNE-N2dTW62IuyLBCZl2eLJlZJyQMSHxEQXiVb8PcCroxOU94pP7fNCkXemz6OCHseu7bw'
token = SANDBOX
TOKEN = token

In [16]:
import yaml
with open('settings.yaml') as f:
    # Load YAML data from the file
    read_data = yaml.load(f, Loader=yaml.FullLoader)

read_data["sandbox"]

't.ZsHS2K9VKfl4Od15pNE-N2dTW62IuyLBCZl2eLJlZJyQMSHxEQXiVb8PcCroxOU94pP7fNCkXemz6OCHseu7bw'

### Исторические данные

In [3]:
client = SandboxClient(token)
base = tink.get_id_base(client)

In [4]:
dfw = base[(base["type"] == "shares") & (base["cur"] == "rub")]
dfw.sort_values("ticker")

,ticker,figi,type,name,cur,lot
431,ABIO,TCS10A0JNAB6,shares,Артген,rub,10
891,ABRD,BBG002W2FT69,shares,АбрауДюрсо,rub,10
1815,AFKS,BBG004S68614,shares,АФК Система,rub,100
1271,AFLT,BBG004S683W7,shares,Аэрофлот,rub,10
1204,AGRO,BBG007N0Z367,shares,РусАгро,rub,1
...,...,...,...,...,...,...
175,WUSH,TCS00A105EX7,shares,Whoosh,rub,1
1538,YAKG,BBG002B298N6,shares,ЯТЭК,rub,10
1250,YDEX,TCS00A107T19,shares,Яндекс,rub,1
472,ZAYM,TCS00A107RM8,shares,МФК Займер,rub,10


In [7]:
from dataload import ReadData

data_reader = ReadData(SandboxClient, token)
data_reader.read_id_base()
data_reader.read_candles( 20)
data_reader.to_df()
data_reader.save('portfolio_prices.csv')

ABIO
ABRD
AFKS
AFLT
AGRO
AKRN
ALRS
AMEZ
APTK
AQUA
ASTR
BANE
BANEP
BELU
BLNG
BSPB
CARM
CBOM
CHMF
CHMK


File /home/samsmu/Data/QBP/Tink/Cache/BBG000BBV4M5/CANDLE_INTERVAL_DAY.meta was not found. Creating default.


CIAN
CNTL


File /home/samsmu/Data/QBP/Tink/Cache/BBG0027F0Y27/CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File /home/samsmu/Data/QBP/Tink/Cache/TCS00A108ZR8/CANDLE_INTERVAL_DAY.meta was not found. Creating default.


CNTLP
DATA


...

WUSH
YAKG


File /home/samsmu/Data/QBP/Tink/Cache/TCS00A107T19/CANDLE_INTERVAL_DAY.meta was not found. Creating default.
File /home/samsmu/Data/QBP/Tink/Cache/TCS00A107RM8/CANDLE_INTERVAL_DAY.meta was not found. Creating default.


YDEX
ZAYM


File /home/samsmu/Data/QBP/Tink/Cache/BBG000QFB4J6/CANDLE_INTERVAL_DAY.meta was not found. Creating default.


ZILLP


## Tinkoff sandbox

#### Открываем новый счет

In [6]:
logging.basicConfig(format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [7]:
from tinkoff.invest.sandbox.client import SandboxClient

"""Example - How to set/get balance for sandbox account.
How to get/close all sandbox accounts.
How to open new sandbox account."""
with SandboxClient(token) as client:
    try:
        client.users.get_info()
        connect = True
    except:
        print("Ошибка подключения!")
        connect = False

if connect:
    with SandboxClient(token) as client:
        # get all sandbox accounts
        sandbox_accounts = client.users.get_accounts()
        print(sandbox_accounts)
    
        # close all sandbox accounts
        for sandbox_account in sandbox_accounts.accounts:
            client.sandbox.close_sandbox_account(account_id=sandbox_account.id)
    
        # open new sandbox account
        sandbox_account = client.sandbox.open_sandbox_account()
        print(sandbox_account.account_id)
    
        account_id = sandbox_account.account_id

2024-11-07 10:30:25,995 INFO:114873b65cc621205f52446b90c32fab GetInfo
2024-11-07 10:30:26,196 INFO:44f642daf692738c4993d7aa4e05e27a GetAccounts
2024-11-07 10:30:26,240 INFO:5cbaac0e21333799fac1f6ab8d25a0aa OpenSandboxAccount


GetAccountsResponse(accounts=[])
6543ea6e-d49e-4e2b-9b95-793fb48f4c6a


#### Add money to Account

In [8]:
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal

def add_money_sandbox(client, account_id, money, currency="rub"):
    """Function to add money to sandbox account."""
    money = decimal_to_quotation(Decimal(money))
    return client.sandbox.sandbox_pay_in(
        account_id=account_id,
        amount=MoneyValue(units=money.units, nano=money.nano, currency=currency),
    )

with SandboxClient(token) as client:
      # add initial 1 000 000 to sandbox account
        print(add_money_sandbox(client=client, account_id=account_id, money=1000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )
        print(
            "money: ",
            float(
                quotation_to_decimal(
                    client.operations.get_positions(account_id=account_id).money[0]
                )
            ),
        )

2024-11-07 10:30:37,830 INFO:8a8a97a0e35decf66098cd9a7a0f04a7 SandboxPayIn
2024-11-07 10:30:37,867 INFO:54ca5f63a47b7f239d4c5700daa40592 GetPositions
2024-11-07 10:30:37,869 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=1000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-11-07 10:30:37,914 INFO:844efd1198f30a760421efd64882722a GetPositions


SandboxPayInResponse(balance=MoneyValue(currency='rub', units=1000000, nano=0))
money:  1000000.0


#### Покупаем базовый портфель

In [8]:
client = SandboxClient(token)
base = tink.get_id_base(client)

2024-11-06 10:43:33,699 INFO:6ae303b28265f8d54d94b5bafdbbdcb1 Shares
2024-11-06 10:43:34,326 INFO:254a3ea47092bf3ba73bd9ab4f2ccac2 Currencies
2024-11-06 10:43:34,486 INFO:6e810a70f8233a05c52b1ac0ba907717 Futures
2024-11-06 10:43:34,913 INFO:2951033180deb7cb52c8b8e66fd93ef3 Bonds


In [9]:
base

,ticker,figi,type,name,cur,lot
0,CNX,BBG000CKVSG8,shares,CNX Resources Corporation,usd,1
1,SLM,BBG000BBCQD7,shares,SLM Corp,usd,1
2,ACLS,BBG000DW34S2,shares,Axcelis Technologies,usd,1
3,AMPH,BBG000BW90S6,shares,Amphastar Pharmaceuticals,usd,1
4,RELY,BBG012C7JST2,shares,Remitly Global,usd,1
...,...,...,...,...,...,...
3480,RU000A103VD8,BBG012YS2TP2,bonds,МСБ-Лизинг 002P выпуск 5,rub,1
3481,RU000A105HJ9,TCS00A105HJ9,bonds,Альфа-Банк 002Р-18,rub,1
3482,RU000A101L96,BBG00T7HZJG4,bonds,ЕвроХим выпуск 7,rub,1
3483,RU000A104735,TCS00A104735,bonds,ГК Пионер-001P выпуск 6,rub,1


In [10]:
# BBG004730ZJ9 - VTBR / BBG004730N88 - SBER
portfolio = {'VTBR':10, 'POSI':20, 'RTKM':30, 'IRKT':20}

for asset in portfolio:
    figi = tink.ticker_to_figi(asset, base)
    print(figi)
    

BBG004730ZJ9
TCS00A103X66
BBG004S682Z6
BBG000FWGSZ5


In [7]:
base[base.lot > 1];

In [11]:
from tinkoff.invest import OrderDirection, OrderType


with SandboxClient(token) as client:
    for asset in portfolio:
        print(asset)
        figi = tink.ticker_to_figi(asset, base)
        qty = portfolio[asset]
        resp = client.orders.post_order(figi=figi,
                    quantity=qty,
                    direction=OrderDirection.ORDER_DIRECTION_BUY,
                    account_id=account_id,
                    order_type=OrderType.ORDER_TYPE_MARKET,)

2024-11-06 10:45:50,151 INFO:9dbf5fb8fc8e7273812a411ccf65b384 PostOrder


VTBR
POSI


2024-11-06 10:45:50,254 INFO:0f12a72eae072562571721432270bcb0 PostOrder
2024-11-06 10:45:50,329 INFO:901a75769335c44f67bc6ab13e3ae52e PostOrder
2024-11-06 10:45:50,434 INFO:08fb0ed4b14fcc5b701567f4893faafc PostOrder


RTKM
IRKT


In [14]:
port = tink.TinkSession(SandboxClient, token)

port.get_portfolio()

2024-11-06 10:47:08,087 INFO:f8fb4de35464cc5dbc238639aec292a4 GetAccounts
2024-11-06 10:47:08,221 INFO:614c49e8590f0b181f6a2843c31f2d7c GetPortfolio


PortfolioResponse(total_amount_shares=MoneyValue(currency='rub', units=126844, nano=800000000), total_amount_bonds=MoneyValue(currency='rub', units=0, nano=0), total_amount_etf=MoneyValue(currency='rub', units=0, nano=0), total_amount_currencies=MoneyValue(currency='rub', units=1873021, nano=242350000), total_amount_futures=MoneyValue(currency='rub', units=0, nano=0), expected_yield=Quotation(units=-1, nano=993300000), positions=[PortfolioPosition(figi='RUB000UTSTOM', instrument_type='currency', quantity=Quotation(units=1873021, nano=242350000), average_position_price=MoneyValue(currency='', units=0, nano=0), expected_yield=Quotation(units=0, nano=0), current_nkd=MoneyValue(currency='', units=0, nano=0), average_position_price_pt=Quotation(units=0, nano=0), current_price=MoneyValue(currency='', units=0, nano=0), average_position_price_fifo=MoneyValue(currency='', units=0, nano=0), quantity_lots=Quotation(units=1873021, nano=242350000), blocked=False, blocked_lots=Quotation(units=0, nan

### Account Operations

In [ ]:
with SandboxClient(token) as client:
    logger.info("orders: %s", client.orders.get_orders(account_id=account_id))
    logger.info(
        "positions: %s", client.operations.get_positions(account_id=account_id)
    )
    logger.info(
        "portfolio: %s", client.operations.get_portfolio(account_id=account_id)
    )
    logger.info(
        "operations: %s",
        client.operations.get_operations(
            account_id=account_id,
            from_=datetime(2023, 1, 1),
            to=datetime(2023, 2, 5),
        ),
    )
    logger.info(
        "withdraw_limits: %s",
        client.operations.get_withdraw_limits(account_id=account_id),
    )

### Закрытие аккаунта

In [16]:
with SandboxClient(token) as client:        
        # add + 2 000 000 to sandbox account, total is 4 000 000
        print(add_money_sandbox(client=client, account_id=account_id, money=2000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )

        # close new sandbox account
        sandbox_account = client.sandbox.close_sandbox_account(
            account_id=sandbox_account.account_id
        )
        print(sandbox_account)


2024-11-06 10:49:58,513 INFO:47cc592c526112097af07aae7ea65a54 SandboxPayIn
2024-11-06 10:49:58,550 INFO:5f9a7b6676a8ddd78a424b6fe7b40e32 GetPositions
2024-11-06 10:49:58,553 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=3873021, nano=240000000)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[PositionsSecurities(figi='BBG004S682Z6', blocked=0, balance=300, position_uid='fe4d90a1-c6e9-478f-a1de-dbab19e29390', instrument_uid='02eda274-10c4-4815-8e02-a8ee7eaf485b', exchange_blocked=False, instrument_type='share'), PositionsSecurities(figi='BBG004730ZJ9', blocked=0, balance=10, position_uid='626d2ab0-3359-4d2d-8366-daedd2a0d000', instrument_uid='8e2b0325-0292-4654-8a18-4f63ed3b0e09', exchange_blocked=False, instrument_type='share'), PositionsSecurities(figi='BBG000FWGSZ5', blocked=0, balance=2000, position_uid='1a583ddf-b4af-426b-890f-2749767a8379', instrument_uid='cfb50a23-2465-497e-bc7e-e4f0e042cf3d', exchange_blocked=False, instrument_type

SandboxPayInResponse(balance=MoneyValue(currency='rub', units=3873021, nano=240000000))
CloseSandboxAccountResponse()


### Get order price

In [ ]:

import logging
import os
from decimal import Decimal

from tinkoff.invest import Client, GetOrderPriceRequest, OrderDirection
from tinkoff.invest.utils import decimal_to_quotation

TOKEN = TOKEN1

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


INSTRUMENT_ID = "TCS00A105GE2"
QUANTITY = 1
PRICE = 230.1



logger.info("Getting Max Lots")
with SandboxClient(TOKEN) as client:
 #   response = client.users.get_accounts()
 #   account, *_ = response.accounts
 #   account_id = account.id

    logger.info(
        "Get pre-trade order commission and price for instrument=%s, quantity=%s and price=%s",
        INSTRUMENT_ID,
        QUANTITY,
        PRICE,
    )
    get_order_price = client.orders.get_order_price(
        GetOrderPriceRequest(
            account_id=account_id,
            instrument_id=INSTRUMENT_ID,
            quantity=QUANTITY,
            direction=OrderDirection.ORDER_DIRECTION_BUY,
            price=decimal_to_quotation(Decimal(PRICE)),
        )
    )

print(get_order_price)


In [41]:
account_id

'9da21a29-1529-4b06-8e5f-7d88cca949c3'

In [ ]:
import logging
import os
from decimal import Decimal

from tinkoff.invest import (
    Client,
    InstrumentIdType,
    StopOrderDirection,
    StopOrderExpirationType,
    StopOrderType,
)
from tinkoff.invest.exceptions import InvestError
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal

with SandboxClient(TOKEN) as client:
#    response = client.users.get_accounts()
#    account, *_ = response.accounts
#    account_id = account.id
    logger.info("Orders: %s", client.orders.get_orders(account_id=account_id))

    figi = "BBG004730ZJ9"  # BBG004730ZJ9 - VTBR / BBG004730N88 - SBER

    # getting the last price for instrument
    last_price = (
        client.market_data.get_last_prices(figi=[figi]).last_prices[0].price
    )
    last_price = quotation_to_decimal(last_price)
    print(f"figi, last price = {last_price}")

    # setting the percentage by which the takeprofit stop order
    # should be set below the last price
    percent_down = 5

    # calculation of the price for takeprofit stop order
    calculated_price = last_price - last_price * Decimal(percent_down / 100)
    print(f"calculated_price = {calculated_price}")

    # getting the min price increment and the number of digits after point
    min_price_increment = client.instruments.get_instrument_by(
        id_type=InstrumentIdType.INSTRUMENT_ID_TYPE_FIGI, id=figi
    ).instrument.min_price_increment
    number_digits_after_point = 9 - len(str(min_price_increment.nano)) + 1
    min_price_increment = quotation_to_decimal(min_price_increment)
    print(
        f"min_price_increment = {min_price_increment}, "
        f"number_digits_after_point={number_digits_after_point}"
    )

    # calculation of the price for instrument which is
    # divisible to min price increment
    calculated_price = (
        round(calculated_price / min_price_increment) * min_price_increment
    )
    print(
        f"let's send stop order at price = "
        f"{calculated_price:.{number_digits_after_point}f} divisible to "
        f"min price increment {min_price_increment}"
    )


### Market Order

In [13]:
with SandboxClient(token) as client:
    trading_status = client.market_data.get_trading_status(
        figi=figi
    )
trading_status.api_trade_available_flag
trading_status.market_order_available_flag

2024-04-03 09:11:12,726 INFO:0ededbd580708669f87e75db1eda44f8 GetTradingStatus


False

In [9]:
from tinkoff.invest import OrderDirection, OrderType
figi = "BBG004730ZJ9"
with SandboxClient(token) as client:
    resp = client.orders.post_order( figi=figi,
                quantity=1,
                direction=OrderDirection.ORDER_DIRECTION_BUY,
                account_id=account_id,
                order_type=OrderType.ORDER_TYPE_MARKET,)

2024-04-03 09:07:12,160 ERROR:18390a18d80f59ef1d8b0a6b70e7a0e6 PostOrder INVALID_ARGUMENT 30079


RequestError: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30079', Metadata(tracking_id='18390a18d80f59ef1d8b0a6b70e7a0e6', ratelimit_limit='200, 200;w=60', ratelimit_remaining=199, ratelimit_reset=49, message='instrument is not available for trading'))

In [55]:
resp

PostOrderResponse(order_id='8b666896-b27d-46e1-bcec-435b5d5f6231', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=232, nano=0), executed_order_price=MoneyValue(currency='rub', units=0, nano=23200000), total_order_amount=MoneyValue(currency='rub', units=232, nano=0), initial_commission=MoneyValue(currency='rub', units=0, nano=116000000), executed_commission=MoneyValue(currency='rub', units=0, nano=116000000), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730ZJ9', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=0, nano=23200000), order_type=<OrderType.ORDER_TYPE_MARKET: 2>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='8e2b0325-0292-4654-8a18-4f63ed3b0e09', order_request_id='', response_metadata=ResponseMetadata(tracking_id='d4f88ec0baaae61b8b3fd5692828b8

In [66]:
base = tink.get_id_base(token)

sess = SandboxClient(token) 

with sess as cl:
    port = cl.operations.get_portfolio(account_id=account_id) 
tink.port_to_df(port, base)

2024-04-03 08:22:30,915 INFO:3b660782159cb02542c10cdfa5550706 Shares
2024-04-03 08:22:32,176 INFO:27fa9439f271f783e285ff053e0a4076 Currencies
2024-04-03 08:22:32,436 INFO:52f29eda572f433d4b4eac4b7b452298 Futures
2024-04-03 08:22:33,798 INFO:c85b8fa40e8996550b72672ee3c5b185 Bonds
2024-04-03 08:22:34,731 INFO:8f26771087d22975c10be9585acb9c0b Etfs
2024-04-03 08:22:35,041 INFO:1b446e3e56694a780af05eb98b175fa5 GetPortfolio


,figi,ticker,name,quantity,price
1,BBG004730ZJ9,VTBR,Банк ВТБ,20000,0.02363
0,RUB000UTSTOM,None,None,1999535,0.00000


In [53]:
with SandboxClient(token) as client:
    statuses = client.market_data.get_trading_statuses(instrument_ids=["BBG004730N88"])
    print(statuses)

2024-04-02 20:12:51,913 INFO:a5543646ba0e0aea5be67619ae261c2a GetTradingStatuses


GetTradingStatusesResponse(trading_statuses=[GetTradingStatusResponse(figi='BBG004730N88', trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_NORMAL_TRADING: 5>, limit_order_available_flag=True, market_order_available_flag=True, api_trade_available_flag=True, instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', bestprice_order_available_flag=True, only_best_price=False)])


## Helper functions

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from datetime import datetime
now = datetime.today().strftime('%Y-%m-%d')


In [16]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]

    return difference


In [31]:
def calculate_portfolio_allocation(weights, total_amount, lot_sizes):
    """
    Рассчитать распределение суммы на веса в портфеле, учитывая лоты.

    Args:
        weights: Список весов акций в портфеле.
        total_amount: Общая сумма, которую нужно распределить.
        lot_sizes: Список размеров лотов для акций.

    Returns:
        Список, где каждый элемент - количество акций, 
        округленное до размера лота.
    """

    # Нормализация весов
    weights = normalize_weights(weights)

    # Распределение суммы
    amounts = [weight * total_amount for weight in weights]

    # Округление до размера лота
    quantities = [round(amount / lot_size, 0) for amount, lot_size in zip(amounts, lot_sizes)]

    # Корректировка с учетом минимального лота
    for i in range(len(quantities)):
        if lot_sizes[i] > 0 and quantities[i] * lot_sizes[i] < total_amount:
            quantities[i] = 1

    # Распределение остатка
    remaining_amount = total_amount - sum(quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes))

    return quantities, remaining_amount


def normalize_weights(weights):
    """
    Нормализовать веса, чтобы они давали в сумме единицу.

    Args:
        weights: Список весов.

    Returns:
        Список нормализованных весов.
    """

    total_weight = sum(weights)
    return [weight / total_weight for weight in weights]


# Пример
weights = [0.4, 0.3, 0.3]
total_amount = 10000
lot_sizes = [1, 10, 1000]

quantities, rem = calculate_portfolio_allocation(weights, total_amount, lot_sizes)

# Вывод
for quantity, weight in zip(quantities, weights):
    print(f"Количество: {quantity}, Вес: {weight:.2%}")

print(f"\nОбщая сумма: {sum(quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes))}")

Количество: 1, Вес: 40.00%
Количество: 1, Вес: 30.00%
Количество: 1, Вес: 30.00%

Общая сумма: 1011


In [21]:
def calculate_portfolio_allocation(weights, total_amount, lot_sizes):
    """
    Рассчитать распределение суммы на веса в портфеле, учитывая лоты.

    Args:
        weights: Список весов акций в портфеле.
        total_amount: Общая сумма, которую нужно распределить.
        lot_sizes: Список размеров лотов для акций.

    Returns:
        Список, где каждый элемент - 
        словарь с информацией о количестве акций, цене за 1 акцию, 
        цене за лот и стоимости позиции.
    """

    # Нормализация весов
    weights = normalize_weights(weights)

    # Распределение суммы
    amounts = [weight * total_amount for weight in weights]

    # Округление до размера лота
    quantities = [round_to_lot_size(amount, lot_size) for amount, lot_size in zip(amounts, lot_sizes)]

    # Расчет стоимости позиции
    position_values = [quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes)]

    # Формирование выходных данных
    portfolio = []
    for quantity, lot_size, position_value in zip(quantities, lot_sizes, position_values):
        portfolio.append({
            "quantity": quantity,
            "price_per_lot": position_value / quantity,
            "position_value": position_value,
        })

    return portfolio


def round_to_lot_size(amount, lot_size):
    """
    Округлить число до размера лота.

    Args:
        amount: Число, которое нужно округлить.
        lot_size: Размер лота.

    Returns:
        Округленное число.
    """

    if lot_size == 1:
        return round(amount)

    return round(amount / lot_size) * lot_size


# Пример
weights = [0.4, 0.3, 0.3]
total_amount = 10000
lot_sizes = [1, 10, 1000]

portfolio = calculate_portfolio_allocation(weights, total_amount, lot_sizes)

# Вывод
for position in portfolio:
    print(f"Количество: {position['quantity']}")
    print(f"Цена за лот: {position['price_per_lot']}")
    print(f"Стоимость позиции: {position['position_value']}")
    print()

print(f"\nОбщая сумма: {sum(position['position_value'] for position in portfolio)}")

Количество: 4000
Цена за лот: 1.0
Стоимость позиции: 4000

Количество: 3000
Цена за лот: 10.0
Стоимость позиции: 30000

Количество: 3000
Цена за лот: 1000.0
Стоимость позиции: 3000000


Общая сумма: 3034000


In [20]:
import json

d = {
    'base': ['FIVE'],
    'momentum': ['GTRK']
}
 
with open('drops.json', 'w') as f:
    json.dump(d, f)

In [22]:
with open('drops.json', 'r') as f:
    d = json.load(f)

In [24]:
from pathlib import Path

Path('asd', 'sd')

WindowsPath('asd/sd')